In [2]:

from db_connector import DB
from Exception import CustomException
import pandas as pd
import numpy as np
from logger import logging
from datetime import datetime

###1)Pull data from raw_data 
    2) Process the Data 
    3) delete the Data
    4) 

In [12]:

class ELT():

    def __init__(self) -> None:
        self.DB = DB()
        self.data = self.DB.pull(filter={},colunm={"_id":0},db="Raw_data",document="Creators") 
        self.df = pd.DataFrame(self.data)

    def ETL(self):

        df = self.df[self.df.error != 'Page not found']
        posts = []
        for i in df["latestPosts"]:
            if type(i) == float:
                continue
            posts = posts + i
        df_post = pd.DataFrame(posts)


        drop_labels_main = ["inputUrl","url","externalUrl","hasChannel",
                    "latestIgtvVideos","joinedRecently","highlightReelCount",
                    "profilePicUrl","externalUrlShimmed","igtvVideoCount"]
        drop_labels_post = ['shortCode', 'url', 'dimensionsHeight', 'dimensionsWidth','displayUrl',
            'images', 'videoUrl', 'childPosts'
            ,'locationName', 'locationId']
        df.drop(drop_labels_main,axis= 1,inplace=True)
        df_post.drop(drop_labels_post,axis = 1,inplace = True)
        df_post.rename({"id":"media_id"},inplace= True)


        df_post.rename(columns = {"id":"media_id"},inplace= True)
        df_post = df_post[df_post["isPinned"] != True]
        df_post["timestamp"] = df_post["timestamp"].apply(datetime.fromisoformat,1)


        df_post = df_post.merge(df[["id","followersCount"]],left_on="ownerId",right_on= "id")

        df_post["ER_per"] = ((df_post["likesCount"] + df_post["commentsCount"])/df_post["followersCount"])*100
        df_post["Reach_per"] = ((df_post["videoViewCount"])/df_post["followersCount"])*100
        df_post["Likes_per"] = ((df_post["likesCount"])/df_post["followersCount"])*100
        df_post["Cmnts_per"] = ((df_post["commentsCount"])/df_post["followersCount"])*100
        df_post["LC_ratio"] = df_post["commentsCount"]/df_post["likesCount"]
        #Creating Different measure of enagement of an acc

        df = df.merge(df_post.groupby(["ownerUsername","ownerId"])
                .agg(avg_likes =("Likes_per","mean"),avg_cmnt =("Cmnts_per","mean"),avg_reach = ("Reach_per","mean"),avg_ER =("ER_per","mean"),avg_LC =("LC_ratio","mean"))
                ,left_on= "id",right_on= "ownerId")

        id_to_hash = {}
        id_to_related = {}

        for i  in df['id']:
            lis = []
            for j in df_post[df_post["ownerId"] == i]["hashtags"]:
                lis = lis + j
            id_to_hash[i] = lis
        temp_lis_1 = []
        temp_lls_2 = []
        for i in id_to_hash.keys():
            temp_lis_1.append(i)
            temp_lls_2.append(id_to_hash[i])
        df_hash = pd.DataFrame({"id":temp_lis_1,"hashtags":temp_lls_2})    
        df = df.merge(df_hash,left_on = "id",right_on = "id")    




        for i in df["id"]:
            lis = []
            for j in df[df["id"] == i]["relatedProfiles"]:
                for k in j:
                    lis.append(k["id"])
            id_to_related[i] = lis
        temp_lis_1 = []
        temp_lls_2 = []    
        for i in id_to_related.keys():
            temp_lis_1.append(i)
            temp_lls_2.append(id_to_related[i])
        df_related = pd.DataFrame({"id":temp_lis_1,"related":temp_lls_2})    
        df = df.merge(df_related,left_on = "id",right_on = "id")
        df_final = df[['date', 'fullName', 'followersCount', 'verified',
        'followsCount', 'private', 'username',
        'isBusinessAccount', 'id', 'businessCategoryName', 'biography',
        'postsCount','avg_likes', 'avg_cmnt',
        'avg_reach', 'avg_ER', 'avg_LC', 'hashtags', 'related']]
        df_post.drop(labels=["mentions","alt","ownerUsername","isPinned","id"],inplace=True,axis=1)
        for j in df_final["username"]:
            to_scan_dict = {"id":j,"priority":1}
            self.DB.push(to_scan_dict,"Clean","To_scan")
        self.DB.push_many(data = df_final.to_dict(orient = "records"),db = "Clean",document = "Creators")
        self.DB.push_many(data = df_post.to_dict(orient = "records"),db = "Clean",document = "Posts")
        return df_final,df_post


In [13]:
E = ELT()
try:

    df_final,df_post = E.ETL()
except CustomException as e:
    raise e


rgrimdesign
drveronicakapur_
jasmeetkapany
abhinandankumar2.0
sunshine.sparrows
tenajaiin
nicoleyoungtech
nailsbydvisy
professorofhow
midlife.muse
blackmotherwound
vinali_bhatnagar
theohiogirljaz
harshhrane
vinit_adwani
accountant_she
talwarisha
benanna_love
kanakmishrax
juhi.bhatt
yourseesha
rivyarai_27
nutritionbyrobyn
mehar.labana
dikshuuu
charan_peri
sachinsins
iam_priyanka_kd
rayrooqma
prakritipavani
siimran_siingh
jinal.j.j
somashribhattacharyyaofficial
neelajolene
shaili_bhattacharjee
soumichakraborty_official
nikitabhardwajofficial_11
debchandrima_official
darshanabanik
vedvika_
simplicityandastarter
satyajeetdubey
soniheena__
shreyaapujari
shinepandey
aisharahmed
aadhya_anand
sanvi_bhagat7
urvisingh143
vishakhapandey510
revathi.05
amandeep_sidhu___
nikilalwani_27
sargun_kaur.luthra
akankshaaryaa
am_akankshasingh
sunakshigrover
reel_ananya
rosie.rapp
anushrisurekhamane
sannidhyabaweja
digitalstreamliner
shubhsharma.in
vaibhavv.ai
digital_vamsi
indianfarmer
ahsaassy_
gayatribhar

C:\Users\rahul\AppData\Local\Temp\ipykernel_19864\4084350526.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(drop_labels_main,axis= 1,inplace=True)
C:\Users\rahul\AppData\Local\Temp\ipykernel_19864\4084350526.py:32: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_post["timestamp"] = df_post["timestamp"].apply(datetime.fromisoformat,1)


In [25]:
df_post.head()

,media_id,type,caption,hashtags,commentsCount,likesCount,videoViewCount,timestamp,ownerId,productType,taggedUsers,time,followersCount,ER_per,Reach_per,Likes_per,Cmnts_per,LC_ratio
0,3331615091575840941,Video,"Did you know that using Figma alone, you can c...","[figma, designguide, dailyinspiration, uiux, u...",60,1982,17826.0,2024-03-25 15:12:29+00:00,57274797898,clips,NaN,15:12:29,191322.0,1.067311,9.317277,1.035950,0.031361,0.030272
1,3326461068803765396,Video,Hey guys! 🍕Wanted to share this creative pizza...,"[figma, designguide, dailyinspiration, uiux, u...",160,7530,142004.0,2024-03-18 12:32:00+00:00,57274797898,clips,NaN,12:32:00,191322.0,4.019402,74.222515,3.935773,0.083629,0.021248
2,3324338076963381657,Video,Dive into a rainbow of flavors with my new smo...,"[figma, designguide, dailyinspiration, uiux, u...",151,19546,224263.0,2024-03-15 14:14:03+00:00,57274797898,clips,NaN,14:14:03,191322.0,10.295209,117.217570,10.216285,0.078925,0.007725
3,3322838046490475921,Video,Check out how easy it is to transform flat pla...,"[figma, designguide, dailyinspiration, uiux, u...",48,2769,24137.0,2024-03-13 12:34:33+00:00,57274797898,clips,NaN,12:34:33,191322.0,1.472387,12.615904,1.447298,0.025089,0.017335
4,3319367867978732674,Video,Dive into the wizarding world with this Harry ...,"[figma, designguide, dailyinspiration, uiux, u...",51,1330,12953.0,2024-03-08 17:39:27+00:00,57274797898,clips,NaN,17:39:27,191322.0,0.721820,6.770262,0.695163,0.026657,0.038346


In [23]:
print((datetime.fromisoformat(df_post["timestamp"][0]).date()))

2024-03-25


In [31]:
datetime(df_post["timestamp"][0])

TypeError: 'Timestamp' object cannot be interpreted as an integer

In [57]:
df_final.to_csv("Creators.csv",index=False)

In [61]:
df_final_json = df_final.to_dict(orient = "records",)

In [66]:
D.push(df_final_json[0],db="Clean",document="Creators")

In [42]:
df_post.head()

,media_id,type,caption,hashtags,commentsCount,likesCount,videoViewCount,timestamp,ownerId,productType,taggedUsers,followersCount,ER_per,Reach_per,Likes_per,Cmnts_per,LC_ratio
0,3331615091575840941,Video,"Did you know that using Figma alone, you can c...","[figma, designguide, dailyinspiration, uiux, u...",60,1982,17826.0,2024-03-25T15:12:29.000Z,57274797898,clips,NaN,191322.0,1.067311,9.317277,1.035950,0.031361,0.030272
1,3326461068803765396,Video,Hey guys! 🍕Wanted to share this creative pizza...,"[figma, designguide, dailyinspiration, uiux, u...",160,7530,142004.0,2024-03-18T12:32:00.000Z,57274797898,clips,NaN,191322.0,4.019402,74.222515,3.935773,0.083629,0.021248
2,3324338076963381657,Video,Dive into a rainbow of flavors with my new smo...,"[figma, designguide, dailyinspiration, uiux, u...",151,19546,224263.0,2024-03-15T14:14:03.000Z,57274797898,clips,NaN,191322.0,10.295209,117.217570,10.216285,0.078925,0.007725
3,3322838046490475921,Video,Check out how easy it is to transform flat pla...,"[figma, designguide, dailyinspiration, uiux, u...",48,2769,24137.0,2024-03-13T12:34:33.000Z,57274797898,clips,NaN,191322.0,1.472387,12.615904,1.447298,0.025089,0.017335
4,3319367867978732674,Video,Dive into the wizarding world with this Harry ...,"[figma, designguide, dailyinspiration, uiux, u...",51,1330,12953.0,2024-03-08T17:39:27.000Z,57274797898,clips,NaN,191322.0,0.721820,6.770262,0.695163,0.026657,0.038346


In [39]:
df_final.columns

Index(['date', 'fullName', 'followersCount', 'verified', 'followsCount',
       'private', 'username', 'isBusinessAccount', 'id',
       'businessCategoryName', 'biography', 'postsCount', 'avg_likes',
       'avg_cmnt', 'avg_reach', 'avg_ER', 'avg_LC', 'hashtags', 'related'],
      dtype='object')

In [43]:
df_final.head()


,date,fullName,followersCount,verified,followsCount,private,username,isBusinessAccount,id,businessCategoryName,biography,postsCount,avg_likes,avg_cmnt,avg_reach,avg_ER,avg_LC,hashtags,related
0,2024-04-11,Rina Grim • UI UX Designer,191322.0,False,83.0,False,rgrimdesign,False,57274797898,Designer,Tips & Tricks⚡️ #ux #ui\nCool ideas for UI/UX ...,86.0,3.152103,0.049945,36.541653,3.202048,0.023226,"[figma, designguide, dailyinspiration, uiux, u...","[61811951761, 49327871552, 62596079754, 116770..."
1,2024-04-11,Dr. Veronica Kapur | Physiotherapist,7094.0,False,107.0,False,drveronicakapur_,False,60206651110,Medical & health,💪🏻 Helping you live a Pain- Free life\n⭐️ Manu...,118.0,1.948830,0.130392,65.509586,2.079222,0.075854,"[physio, physiomemes, trendingreel, physio, my...","[3045038584, 45292432751, 13584853270, 8592010..."
2,2024-04-11,Jasmeet Kapany | Hair & Makeup,74184.0,True,1367.0,False,jasmeetkapany,False,1422204916,Makeup Artist,📝.. Pro Makeup Course 1-5 May,891.0,2.225210,0.028533,83.825353,2.253743,0.016498,"[makeup, makeupartist, explore, trending, brid...","[3999953524, 484127358, 7281932019, 5694087198..."
3,2024-04-11,Abhinandan Kumar,7740.0,True,941.0,False,abhinandankumar2.0,False,5436755420,Digital creator,👉 𝗖𝗼𝗻𝘁𝗲𝗻𝘁 𝗰𝗿𝗲𝗮𝘁𝗼𝗿 | 𝗦𝗽𝗲𝗮𝗸𝗲𝗿 | 𝗧𝗿𝗮𝗶𝗻𝗲𝗿\n👉 𝗙𝗼𝘂𝗻𝗱...,409.0,4.820557,0.579960,45.620155,5.400517,0.150522,"[affiliatemarketing, skillindia, skillguruu, l...","[57883738943, 52719809177, 17166585057, 640912..."
4,2024-04-11,Rosie Elizabeth | Our YYC Alberta Life + Travel,21471.0,True,1327.0,False,sunshine.sparrows,False,492697243,Personal blog,Good Vibes ➳ Childhood Magic ➳ Outdoor Adventu...,1039.0,1.069272,0.309720,1.810000,1.378992,0.367763,"[52hikechallenge, 52hikechallenge2023, springd...",[]
